# HANDLING MISSING DATA USING SCIKIT-LEARN LIB
This part is based on the 1.2 section of the introduction-to-scikit-learn-video.ipynb = video file.

But it is more appropriate that this is based on section 1.2 from introduction-to-scikit-learn.ipynb = example file.

In the past I have handle missing variables using Pandas functions.<br>
Sckikit-Learn also has [method called SimpleImputer()](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn-impute-simpleimputer). This method is defined as imputation transformer for completing missing values. This is part of the [sklearn.impute](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.impute) library. <br><br>
Now before I begin training to fill the missing data I need the data frame which has missing data in it. Thus I need to use pandas libs once again and import the CSV data into data frame in this notebook.

In [1]:
# import pandas to get the csv with missing data in it 
# the data imported is the same as in the scikit-learn both video and example files.
import pandas as pd
# set the random seed
import numpy as np 
np.random.seed(42)
# import the data
car_sales = pd.read_csv("../data/car-sales-extended-missing-data.csv")
car_sales

,Make,Colour,Odometer (KM),Doors,Price
0,Honda,White,35431.0,4.0,15323.0
1,BMW,Blue,192714.0,5.0,19943.0
2,Honda,White,84714.0,4.0,28343.0
3,Toyota,White,154365.0,4.0,13434.0
4,Nissan,Blue,181577.0,3.0,14043.0
...,...,...,...,...,...
995,Toyota,Black,35820.0,4.0,32042.0
996,NaN,White,155144.0,3.0,5716.0
997,Nissan,Blue,66604.0,4.0,31570.0
998,Honda,White,215883.0,4.0,4001.0


In order to see better on how much data is missing from the car_sales data frame I can use the [pandas data frame isna function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isna.html).  

In [2]:
# the pandas data frame is car_sales
car_sales.isna()

,Make,Colour,Odometer (KM),Doors,Price
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
995,False,False,False,False,False
996,True,False,False,False,False
997,False,False,False,False,False
998,False,False,False,False,False


The data becoming True and False. The True means at this location the missing data is exist. However, I want to find out how much missing data exist inside the data frame. To do this I need [pandas data frame sum function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sum.html)

In [3]:
# calculate how many missing data inside the car_sales data frame
car_sales.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

## 1.a. Strategy on Missing Price Data
Now, from the sum of missing data there are 50 missing Price data. Price is the variable we want to predict using machine learning model. Thus I need to be more careful when there is missing data for Price. The wisest option is to just drop the rows which Price data are missing. <br><br>
Granted that there are 50 rows will be dropped but it is 50 rows from 1000 rows. Thus it counts only 5% of the whole data in the data frame. <br><br>
Moreover, maybe I can also reduce rows which also has missing data for other features. <br><br>
In order to be able to selectively drop rows which has Price missing data I need to use another [pandas function data frame dropna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html?highlight=dropna) 


In [4]:
# Drop the rows which has "Price" feature (column) missing data
car_sales.dropna(0, subset=["Price"], inplace=True)
# check how many missing data persist after we drop the missing data rows on "Price" feature.
car_sales.isna().sum()

Make             47
Colour           46
Odometer (KM)    48
Doors            47
Price             0
dtype: int64

In [5]:
car_sales.shape

(950, 5)

Now from the sum of missing values above all "Price" feature missing values are alreeady being dropped. This is inferred in the sum of missing data in the Price column is now 0. As I drop the rows with Price missing data the whole data frame decrease in number of rows by 50 and thus the shape now becomes (950, 5).<br><br>
Unfortunately, the drop process does not significantly reduces the rows with missing data for other features. This will be a problem if I decided to drop all of those rows with missing data within them. The number of rows dropped might be significantly big that it will affect the model training and testing validity later on.

CONCLUSION: THIS IS NOT THE RECOMMENDED SOLUTION!

Now I know why we don't use the same secton numbering as in the original course material. This is because we also included (or added) methods from previous Pandas course on how to handle missing data value(s)

## 1.b. Strategy to Transform Other Features Missing Data
As mentioned above, the number  of missing data in other features might be too big to drop. Thus I will need to find another way to transform rather than drop those missing data. As mentioned at the top I will need [method called SimpleImputer()](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn-impute-simpleimputer). <br><br>
Afterwards, I need to transform the columns which has its missing data transformed. To do this I wil use the [sklearn.compose.ColumnTransformer fucntion](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html). CAUTION: This column transformer has complicated parameter rules. Divide and assign complicated objects to variables in advance will certainly help with argument passing to the parameters and prevent syntax errors.<br><br>
As for the data_frame itself I will need them to be separated first to X as the data frame of features which drop the whole "Price" column and y as the label data frame,

NOTE: this is what is different with the original data in the course:
1. In this file we use depleted data since I have dropped rows that are missing price data. 
1. In this depleted data I just directly use SimpleImputer method from Scikit Learn and not trying the data imputation technique from Pandas called fillna.
1. To be fair I already use fillna back when I learn Pandas, Maybe in this case I think this was uncessary?

In [6]:
# separate the data frame to X and y
y = car_sales["Price"]
X = car_sales.drop("Price", axis=1)
# axis = 1 means drop the column not the row
# here is the result:
X.head(), y.head()

(     Make Colour  Odometer (KM)  Doors
 0   Honda  White        35431.0    4.0
 1     BMW   Blue       192714.0    5.0
 2   Honda  White        84714.0    4.0
 3  Toyota  White       154365.0    4.0
 4  Nissan   Blue       181577.0    3.0,
 0    15323.0
 1    19943.0
 2    28343.0
 3    13434.0
 4    14043.0
 Name: Price, dtype: float64)

The SimpleImputer class provides basic strategies for imputing missing values. Missing values can be imputed with a provided constant value, or using the statistics (mean, median or most frequent) of each column in which the missing values are located. This class also allows for different missing values encodings.
<br><br>
For each of this feature there will be different strategy on handlind their missing data:
1. Make and Colour data are cathegorical data, thus their missing value will be substituted wiht constant value "N/A" for not available.
1. Doors also cathegorical data although it is numerical. So their missing data will be substituted with constant value which is also number = 4. Meaning all missing data on doors will be assumed as 4 doors car.
1. Odometer is a numerical data, thus its missing data will be substituted with the mean of all available data in Odometer feature.

Thus I need to set each strategy using SimpleImputer function and assign each of them to a variable.<br><br>
Then I need to separate the transform into two type:
1. fit_transform for train X data since from the train data we learn the pattern and fit the mean if any.
1. transform for test X data to just implement the fit of mean from train data in test data features.

Thus, the X and y need to be split to X_train, X_test, y_train, y_test using the [train test split functio](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn-model-selection-train-test-split).

In [7]:
# import the libs to fill missing data and then transform the column 
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
# using the SimpleImputer for each strategy
cath_imputer = SimpleImputer(strategy="constant", fill_value="n/a")
door_imputer = SimpleImputer(strategy="constant", fill_value=4)
mean_imputer = SimpleImputer(strategy="mean")
# now group the features according to its imputer strategy
cath_feat = ["Make", "Colour"]
door_feat = ["Doors"]
mean_feat = ["Odometer (KM)"]
# split data frame to train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)
# now set the transformer using the ColumnTransformer
imputer = ColumnTransformer([
    ("cathergorical", cath_imputer, cath_feat),
    ("doors", door_imputer, door_feat), 
    ("orometer", mean_imputer, mean_feat)
])
# NOTE: WE CAN LEAVE THE reminder = "drop" since we transform all features thus nothing to drop!
# now fill each the train data and test data according to the strategy
fill_X_train = imputer.fit_transform(X_train)
fill_X_test = imputer.transform(X_test)

# Now check the resut:
fill_X_train

array([['Honda', 'White', 4.0, 71934.0],
       ['Toyota', 'Red', 4.0, 162665.0],
       ['Honda', 'White', 4.0, 42844.0],
       ...,
       ['Toyota', 'White', 4.0, 196225.0],
       ['Honda', 'Blue', 4.0, 133117.0],
       ['Honda', 'n/a', 4.0, 150582.0]], dtype=object)

Here the result of the ColumnTransformer function is an array. This makes it hard to proof that all missing data are already transformed. To proof that there are no missing data inside the data frame both for fill_X_train and fill_X_test we need to transform both arrays to data frame.

In [8]:
# make pandas data frame from the fill train and test data
car_sales_fill_train = pd.DataFrame(fill_X_train, columns=["Make", "Colour", "Doors", "Odometer (KM)"])
car_sales_fill_test = pd.DataFrame(fill_X_test, columns=["Make", "Colour", "Doors", "Odometer (KM)"])
# now let's see the head result
car_sales_fill_train.head(), car_sales_fill_test.head()

(     Make Colour Doors Odometer (KM)
 0   Honda  White   4.0       71934.0
 1  Toyota    Red   4.0      162665.0
 2   Honda  White   4.0       42844.0
 3   Honda  White   4.0      195829.0
 4   Honda   Blue   4.0      219217.0,
      Make Colour Doors Odometer (KM)
 0  Toyota   Blue   4.0       99761.0
 1  Toyota  Black   4.0       17975.0
 2   Honda   Blue   4.0      197664.0
 3  Nissan  Green   4.0      235589.0
 4   Honda  Black   4.0      231659.0)

In [9]:
# now validate and compare the number of missing values from the original to the latest fill
car_sales.isna().sum(), car_sales_fill_train.isna().sum(), car_sales_fill_test.isna().sum()

(Make             47
 Colour           46
 Odometer (KM)    48
 Doors            47
 Price             0
 dtype: int64,
 Make             0
 Colour           0
 Doors            0
 Odometer (KM)    0
 dtype: int64,
 Make             0
 Colour           0
 Doors            0
 Odometer (KM)    0
 dtype: int64)

Now all the missing data are filled on both test and train sides. Now the data are ready for next step which is transforming cathegorical data to numerical data to make it valid to build machine learning model parameters.

<br><br>
## 2. Transforming Cathegorical Data to Numerical Data as Preparation for Data Modeling
To transform cathegorical data to numerical data I need to use [ONeHotEncoder function](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn-preprocessing-onehotencoder). Then to transform the column as we already split between train and test I will need to fit transform the train and transform the test.
<br><br>
As for the Transformer we already imported the ColumnTransformer. Just use that to transfom using the encoder.

In [10]:
# import the oneHotEncoder from Scikit Learn libs
from sklearn.preprocessing import OneHotEncoder
encode = OneHotEncoder()
# I will leave the parameters to default in OneHotEncoder since it is safe to use them all.
# prepare the columns (features) need to be transformed
feat_transformed = ["Make", "Colour", "Doors"]
trans_encode = ColumnTransformer([
    ("encode_data", encode, feat_transformed)
])
# now we encode our train filled data (we use fit transform to be used later on the filled test data)
encode_X_train = trans_encode.fit_transform(fill_X_train)

ValueError: Specifying the columns using strings is only supported for pandas DataFrames

### ERROR: 
The transform call using column (feature) name only applicable if the parameter to be transformed is a data frame. Meanwhile the fill_X_train is an array. There are two solutions to solve this error:
1. change fill_X_train with car_sales_fill_train data frame
1. change the feat_transformed variable to [1, 2, 3] which those are the index of the column if the array is table. 

Let's try the second solution as the first one is basically the same as the previous session

In [11]:
feat_transformed = [1,2,3]
trans_encode = ColumnTransformer([
    ("encode_data", encode, feat_transformed)
], remainder='passthrough')
# now we encode our train filled data (we use fit transform to be used later on the filled test data)
encode_X_train = trans_encode.fit_transform(fill_X_train)
encode_X_train.toarray()

ValueError: For a sparse output, all columns should be a numeric or convertible to a numeric.

Well that's not what I am expected. 

ValueError: could not convert string to float: 'Nissan'

The above exception was the direct cause of the following exception:

ValueError: For a sparse output, all columns should be a numeric or convertible to a numeric.

But if I set the reminder parameter back to default = 'drop' this error will not be invoked but the result is not what I am expected.

In [12]:
feat_transformed = [1,2,3]
trans_encode = ColumnTransformer([
    ("encode_data", encode, feat_transformed)
])
# now we encode our train filled data (we use fit transform to be used later on the filled test data)
encode_X_train = trans_encode.fit_transform(fill_X_train)
encode_X_train.toarray()[0]

array([0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

The array result is not the one I expected. Since all the 0s are not representing the data we want. Still the first solution is best for this case. The additional task of makin data frame from the array filled for the missing data is worth the step for result validity.

In [13]:
feat_transformed = ["Make", "Colour", "Doors"]
trans_encode = ColumnTransformer([
    ("encode_data", encode, feat_transformed)
], remainder='passthrough')
# now we encode our train filled data (we use fit transform to be used later on the filled test data)
encode_X_train = trans_encode.fit_transform(car_sales_fill_train)
encode_X_train.toarray()[0]

array([0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
       0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 7.1934e+04])

In [14]:
encode_X_test = trans_encode.transform(car_sales_fill_test)
encode_X_test.toarray()[0]

array([0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
       0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 9.9761e+04])

Both encode_X_train and encode_X_test are already transformed to conver categorical data to numerical data. It is confirmed as the 0s and 1s are representing the all categorical data. The last non 0 or 1 is the int type data which is the Odometer data.

Now we are ready to process further to develop model for machine learning.

## 3 Build Model for Machine Learning using the Filled and Encoded Data
After filling the missing data and then encode categorical data to numerical data, finally our data are ready to be used to develop machine learning model. To do this I need to use prediction libs which is included in the [ensemble methods](https://scikit-learn.org/stable/modules/ensemble.html). I will use the [Random Forrest Regressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn-ensemble-randomforestregressor)

In [15]:
# import the ensemble 
from sklearn.ensemble import RandomForestRegressor
# put the regressor
model = RandomForestRegressor()

# make the model train.
model.fit(encode_X_train, y_train)
model.score(encode_X_test, y_test)


0.21735623151692096

Must say the result is not really consistent. However, I need to clear all outputs, restart kernel and re run all cells. 

CAUTION: ERRORS ABOVE MEANS THE NEXT CELLS MUST BE RUN MANUALLY ONE BY ONE IN ORDER TO GIVE THE OPTIMUM RESULT!!!

# SUMMARY
This is part of the step 1.2 which is handling missing data in the data inputs. The main problem here is the numbering of section is not the same as the original course files. 

However, to be fair this file also contains previous lessons on how to handle missing data using Pandas. Which I think in this case serves the point of remembering old lessons. 